In [3]:
import sys
print(sys.version)
print(sys.path)

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]
['/home/jb/workspace/HLAVO/notebooks/Richards', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/jb/workspace/jupyter_notebooks/jupy-env/lib/python3.10/site-packages']


In [28]:
import pde
import numpy as np
import dataclasses

@dataclasses.dataclass
class Hydraulic:
    th_s: float      # porosity
    th_0: float      # residual water content
    k_s: float       # saturated hydraulic permeability [m/s]
    lmbd: float      # inverse of entry pressure head [1/m]

    def kr_h(self, h):
        H = np.minimum(0.0, h)
        return np.exp(self.lmbd * H)

    def mu_h(self, h):        
        return np.where(h < 0.0,
            np.exp(self.lmbd * h) / self.lmbd,
            h / self.lmbd)     

@dataclasses.dataclass
class RichardsPDE(pde.PDEBase):
    hydraulic: Hydraulic
    flux_top: float  # boundary flux (positive = infiltration) [m/s]
    h_bot: float     # bottom pressure head [m]
    #relative saturation at bottom th = t_0 +(th_s - th_0) * th_r             

    @property
    def a(self):
        m = self.hydraulic
        return m.k_s/(m.th_s - m.th_0)/m.lmbd

    @property
    def b(self):
        m = self.hydraulic
        return m.k_s/(m.th_s - m.th_0)        

    @property
    def mu_bot(self):        
        return self.hydraulic.mu_h(self.h_bot)


    def evolution_rate(self, state, t=0):
        #bc_top = {"type": "mixed", 
        #          "value": - self.hydraulic.lmbd,
        #         "const": self.flux_top / self.hydraulic.k_s}   # d_z mu + "value" * mu = "const"
        
        bc_top = {"value": self.hydraulic.mu_h(-1)}
        bc_bot = {"value": self.mu_bot}
        bc = [bc_top, bc_bot]
        grad_z = state.gradient(bc=bc)[0]
        laplace_z = state.laplace(bc=bc)
        return self.a * laplace_z #- self.b * grad_z

    
L = 1
N = 100
grid = pde.CartesianGrid([[0, L]], N)
init_state = pde.ScalarField.from_expression(grid, "-1")
model = Hydraulic(
    th_s=0.3,
    th_0=0,
    k_s=0.01,
    lmbd=1)    

eq = RichardsPDE(
    hydraulic=model,
    flux_top=0.001,
    h_bot=-1
    )

storage = pde.MemoryStorage()
result = eq.solve(init_state, t_range=(0,1), dt=0.001, method='implicit', tracker=["progress", storage.tracker(1)])
pde.plot_kymograph(storage)


  0%|          | 0/1 [00:00<?, ?it/s]

Implicit Euler step did not converge after 100 iterations at t=0 (error=1.28611e+22)


ConvergenceError: Implicit Euler step did not converge.